In [1]:
import logging

from automata.cli.commands import reconfigure_logging
from automata.config.config_base import AgentConfigName
from automata.config.openai_config import OpenAIAutomataAgentConfigBuilder
from automata.agent.openai_agent import OpenAIAutomataAgent
from automata.singletons.dependency_factory import dependency_factory
from automata.singletons.py_module_loader import py_module_loader
from automata.tools.agent_tool_factory import AgentToolFactory

logger = logging.getLogger(__name__)
reconfigure_logging("DEBUG")

py_module_loader.initialize()

Loading modules with root path: /Users/nolantremelling/automata/automata/core/../.. and py path: /Users/nolantremelling/automata/automata/core/../../automata


In [2]:
# Construct the set of all dependencies that will be used to build the tools
toolkit_list = ["document-oracle","py-reader"]
tool_dependencies = dependency_factory.build_dependencies_for_tools(toolkit_list)

Building dependencies for toolkits ['document-oracle', 'py-reader']...
Building symbol_search...
Creating dependency symbol_search
Creating dependency symbol_graph
Creating dependency symbol_code_embedding_handler
Anonymized telemetry enabled. See https://docs.trychroma.com/telemetry for more information.
Successfully imported ClickHouse Connect C data optimizations
Successfully import ClickHouse Connect C/Numpy optimizations
Using python library for writing JSON byte strings
loaded in 971 embeddings
loaded in 1 collections
Starting component System
Starting component Posthog
Starting component PersistentDuckDB
Starting component LocalAPI
collection with name automata already exists, returning existing collection
Creating dependency embedding_similarity_calculator
Building symbol_doc_embedding_handler...
Creating dependency symbol_doc_embedding_handler
Anonymized telemetry enabled. See https://docs.trychroma.com/telemetry for more information.
loaded in 791 embeddings
loaded in 1 colle

In [3]:
# Build the tools
tools = AgentToolFactory.build_tools(toolkit_list, **tool_dependencies)

In [4]:
# Build the agent config
config_name = AgentConfigName("automata-main")

agent_config = (
    OpenAIAutomataAgentConfigBuilder.from_name(config_name)
    .with_tools(tools)
    .with_model("gpt-3.5-turbo")
    .with_max_iterations(3)
    .build()
)

Creating dependency symbol_rank
Creating dependency subgraph
Pre-computing bounding boxes for all rankable symbols
Finished pre-computing bounding boxes for all rankable symbols in 2.469158887863159 seconds
Building the rankable symbol subgraph...
100%|██████████| 791/791 [00:06<00:00, 115.68it/s]
Built the rankable symbol subgraph


In [5]:
import textwrap
instructions = textwrap.dedent('''
Provide a markdown python snippet, which can validly executed by `exec`. When ran, the snippet produces a valid instance of an openai agent config in the local variable `x`.
''')
agent = OpenAIAutomataAgent(instructions, config=agent_config)

Setting up agent with tools = [OpenAITool(function=<bound method DocumentOracleToolkitBuilder._get_best_match of <automata.experimental.tools.builders.document_oracle_builder.DocumentOracleOpenAIToolkitBuilder object at 0x111d45300>>, name='document-oracle', description="The DocumentOracleToolkitBuilder is a tool that translates a given natural language query into relevant context by finding the most semantically similar symbol's documentation in a Python codebase. It uses SymbolSearch and EmbeddingSimilarityCalculator to identify this symbol. The tool then returns the corresponding class documentation, providing valuable context for the query.", coroutine=None, properties={'query': {'type': 'string', 'description': 'The query string to search for.'}}, required=['query'], openai_function=<automata.llm.providers.openai_llm.OpenAIFunction object at 0x1297cdf60>), OpenAITool(function=<bound method PyReaderToolkitBuilder._run_indexer_retrieve_code of <automata.tools.builders.py_reader_buil

In [6]:
# Run the agent
result = agent.run()


------------------------------------------------------------------------------------------------------------------------
Latest Assistant Message -- 

Approximately 3041 tokens were consumed prior to completion generation.


Function Call:
call_termination

Arguments:
{
  "result": "```python\nimport openai\n\nclass OpenAIAgentConfig:\n    def __init__(self):\n        self.model = \"gpt-3.5-turbo\"\n        self.max_tokens = 100\n        self.temperature = 0.8\n        self.stop_sequence = \"\\n\"\n        self.n = 1\n        self.logprobs = 0\n\nx = OpenAIAgentConfig()\n```"
}

 

Approximately 3045 tokens were after adding the latest message.

------------------------------------------------------------------------------------------------------------------------
Latest User Message -- 
OpenAIChatMessage(role=user, content=```python
import openai

class OpenAIAgentConfig:
    def __init__(self):
        self.model = "gpt-3.5-turbo"
        self.max_tokens = 100
        self.temperature = 0.8
        self.stop_sequence = "\n"
        self.n = 1
        self.logprobs = 0

x = OpenAIAgentConfig()
```, function_call=None)

Approximately 3134 tokens were after adding the latest message.

------------------------------------------------------------------------------------------------------------------------


In [7]:
# Print the result
print(f"Result:\n{result}")

Result:
```python
import openai

class OpenAIAgentConfig:
    def __init__(self):
        self.model = "gpt-3.5-turbo"
        self.max_tokens = 100
        self.temperature = 0.8
        self.stop_sequence = "\n"
        self.n = 1
        self.logprobs = 0

x = OpenAIAgentConfig()
```


In [8]:
result='```python\nfrom automata.config.openai_agent import OpenAIAutomataAgentConfig\n\nx = OpenAIAutomataAgentConfig()\nx.setup()```'
result='```python\n# We need to first import PyReader\nfrom automata.code_parsers.py.py_reader import PyReader\n\n# Next we create an instance of PyReader and assign it to the variable x\nx = PyReader()\n```'
cleaned_result = result.split('```python\n')[1].replace('```','')
print(cleaned_result)

# We need to first import PyReader
from automata.code_parsers.py.py_reader import PyReader

# Next we create an instance of PyReader and assign it to the variable x
x = PyReader()



In [9]:
exec(cleaned_result)

In [10]:
from automata.eval import (
    AgentEval,
    AgentEvalSetLoader,
    AgentEvaluationHarness,
    CodeWritingEval,
    OpenAIFunctionEval,
    CodeWritingAction
)
CodeWritingAction._extract_snippet(result)

Popen(['git', 'version'], cwd=/Users/nolantremelling/automata/examples, universal_newlines=False, shell=None, istream=None)
Popen(['git', 'version'], cwd=/Users/nolantremelling/automata/examples, universal_newlines=False, shell=None, istream=None)


'\n# We need to first import PyReader\nfrom automata.code_parsers.py.py_reader import PyReader\n\n# Next we create an instance of PyReader and assign it to the variable x\nx = PyReader()\n'